In [1]:
import numpy as np
import pandas as pd
import os

%matplotlib qt
import matplotlib.pyplot as plt
import matplotlib as mp

from m_gncspline import global_natural_spline

In [2]:
import dill

with open('smooth_pos_v8.pkl', 'rb') as var:
    sm_pos = dill.load(var)
    
with open('smooth_vel_v8.pkl', 'rb') as var:
    sm_vel = dill.load(var)

with open('smooth_acc_v8.pkl', 'rb') as var:
    sm_acc = dill.load(var)
    
with open('smooth_times_v8.pkl', 'rb') as var:
    d_times = dill.load(var)
    
with open('raw_dictionaries_v9.pkl', 'rb') as var:
    all_imported = dill.load(var)

# Torque analysis process

## Step 1: Load processed position data and corrected marker spacings lists.

Spline fitting requires that there be no missing markers in the frame the spline is being fit to. Trials with any markers missing from the whole trial were updates in Notebook 1 - Data Processing. The resulting updated marker spacings (in cms) account for this. 

In [3]:
marks_s = [all_imported[i]['cms'] for i in np.arange(len(all_imported))] #make a list of all marker spacings

## Step 2: Build lists for each trial of the relevant meta data

To calculate torque, I need to know the frame to calculate at (i.e. the transition frame, or if there is no transition frame, the landing frame). I also need the snake's mass, length, and the size of the gap. 

In [4]:
#build gap sizes, snake number, trial number, svl, and mass lists
gs_m = []
gs_svl = []
sn_id = []
trial_ns = []
    
mass = []
size = []
lands = []
frame_lists = []

for i in np.arange(len(all_imported)):
    
    d = all_imported[i]
    gs_m.append(d['gs_m'])
    gs_svl.append(d['gs_%'])
    sn_id.append(d['ID'])
    trial_ns.append(d['tn'])
    size.append(d['svl']/100) #change cm to m

    mass.append(d['mass']/1000) #change g to kg
    lands.append(int(d['lf']))
    frame_lists.append(d['fn'])

# Step 3: Build transitions frames list

I  now use the visually identified transition frame and the frame index to select the appropriate row index from the csv (the frame numbers, visually identified, do not match up with the row number in most trials as the data has been cropped to the ROI). 

In [5]:
#build new transition frames list
drops = []

cwd = os.getcwd()
meta_path = cwd+'/R files/Summary Datasets/reference_material.csv'
metadata = pd.read_csv(meta_path, delimiter=",")

for i in np.arange(len(sm_pos)):
    trial = all_imported[i]['tn']
    row = metadata[metadata.Trial==trial]

    drop = pd.to_numeric(row.DP.values[0])
    drops.append(drop)

# Define the function

See the documentation here: https://docs.google.com/document/d/1o0ay1PH9b0CNwD_9TEzWPKP6KrioOhOAEUtVL2UPqY4/edit?usp=sharing for a complete description of how this function works. 

In [6]:
def torque(snake,spaces,SVL,mass_total):
    """ determine the torque on the snake in a given frame
    
    Parameters
    ----------
    snake: (m) array of showing a single frame of smoothed, gap-filled snake data from a given trial, mmarks x 3dimensions
    spaces: spacing of markers on the snake, including distance from nosetip to first marker, in m
    SVL: SVL in m
    mass: snake's mass in kg
    
    Returns
    -------
    """

    nmark, dim = np.shape(snake)

    ### Section 1: clean up the snake data
    del_list = []
    n_snake = np.copy(snake)
    n_sp = np.array(spaces[1:]) #cut out the space from nosetip.

    for t in np.arange(len(n_sp)+1):
        mark = n_snake[t,0]
        if np.isnan(mark): #make a list of markers to delete - delete ones with nans. This should never be the head!
            del_list.append(t)

    for l in reversed(del_list):
        n_snake=np.delete(n_snake,l,0) #delete the rows in the snake array
        if l==len(n_sp): #if the index corresponds to the last marker in the spacing array, just delete it
            n_sp=np.delete(n_sp,l-1)
            
        else:
            n_sp[l]=n_sp[l-1]+n_sp[l] #if the index corresponds to some other marker, get the appropriate spacing
            n_sp=np.delete(n_sp,l-1) #and then delete.

    ### Step 2: make the spline
    out = global_natural_spline(n_snake,n_sp,1000) #fit a spline
    r, dr, ddr, dddr, ts, ss, seg_lens, lengths_total, idx_pts = out    
    r_m = r.copy()
      
    density_df = pd.read_csv(cwd+'/R files/Summary Datasets/snake_density.csv', index_col=0)
    s_rho, body_rho = density_df.values.T
    
    density_body = np.interp(ts / SVL, s_rho, body_rho)
    mass_body = mass_total * density_body / density_body.sum()
    
    # find where X > 0 (the body is in the gap; all position data is in a origin-centered coordinate system)
    idx_gap = np.where(r[:, 0] > 0)[0]
    r_gap = r_m[idx_gap]
    
    # distance along the body, reversed because head will have largest moment
    nbody = len(r_gap)
    weight = np.zeros((nbody, 3))
    weight[:, 2] = -mass_body[idx_gap] * 9.81  # N
    
    torques = np.cross(r_gap, weight)  # Nm
    torque_total = torques.sum(axis=0)
    
    return torque_total

# Step 4: Build behavior lists

Making a list of the behaviors in all the trials that the torque code operates on. This allows me to 1) graph the results color-coded by behavior and 2) check that all the non-cantilever trials have drop frames defined. If there is a non-cantilever trial that doesn't have a drop frame, that means there is a mismatch between how I have visually detected behavior and how the behavior code has detected behavior, which needs to be fixed. 

In [7]:
#build behavior lists
bpd = pd.read_csv(cwd+'/R files/Summary Datasets/bdata.csv')

In [8]:
code = bpd['beh']

# Step 5: Run the code on all trials

First, gather relevant metadata for each trial (the trial number, snake ID, behavior code, spacing of markers, landing frame and drop frame, SVL, gap size, etc.) Then: 

## Section 1: Safety checks
Looks for marker spacings lists that don't match the position data shape, no defined origin (needed to determine loop depth and therefore the behavior). 

## Section 2: Determine frame of interest
Using the behavior, determine whether to use the landing frame (cantilevers) or the drop frame (all other behaviors) as the frame of interest at which to calculate torque. Also checks whether there is a mistmatch between visually and code defined behavior. 

## Section 3: Convert frame number to row number
Use the frame index file and the frame number (visually detected) to translate either the drop frame number or the landing frame number (depending on behavior) into a row number. This row is the frame in which the torque will be calculated. 

## Section 4: Additional check
Checks if the head is nan in the frame of interest. Torque cannot be calculated if we don't know where the head is. 

## Section 5: Calculate torques
Use the torque function defined above to calculate the torque vector acting on the snake. X points from the origin to the target branch (unless no target marker exists, in which case it does so approximately), Z is vertical, Y is left/right. The code then saves a list of 1) these torque vectors, 2) the trial numbers that were usable (e.g. not including trials where the head is nan or no origin branch), 3) the normalized torque values (by body weight and length), 4) the gap sizes, 5) the snake IDs, 5) the index of the frame where torque was calculated and 6) the position data for the trials used. It also maintains a list of the excluded trials. Most of these values are useful for troubleshooting -- the main results of interest are the torque values (normalized and otherwise) and corresponding snake IDs and gap sizes. 

In [9]:
trial = []
torques = []
t_normed = []
t_indices = []
gap_svl = []
gap_m = []
sneks = []
t_pos = []
excluded = []
beh_c = []

for i in reversed(np.arange(len(sm_pos))):
    snake = sm_pos[i]/1000.0 #convert mm to m
    A,B,C = np.shape(snake)
    
    spaces = np.array(marks_s[i])/100 #convert cm to m
    mg = mass[i]
    dpi = drops[i]
    lfi = lands[i]
    f_index = frame_lists[i]
    beh_code = code[i]
    tn = trial_ns[i]
    gap_size = gs_svl[i] 
    svl = size[i]
    ID = sn_id[i]
    
    oe = all_imported[i]['oe'] #checking the original record of the OE 
                            #to exclude trials where the origin end was not recorded correctly. 
       
    ###Section 1: Safety Checks!
    if not B==len(spaces):
        print('Marker spacings list does not match position data shape')
        break
        
    elif np.isnan(oe).any():
        print(str(tn)+': Origin end not defined')
        excluded.append(tn)
        continue
    
    ## Section 2: Check behavior and define frame of interest
    elif np.isnan(dpi):
        if not beh_code == 0:
            excluded.append(tn)
            print(str(tn)+': Mismatch between DPI and BC')
            continue
        
        dpi = lfi #if it's a cantilever, use the last frame (landing frame) to measure torque instead of DPI. 
    
    ## Section 3: convert the frame number (saved in the dictionary) to an index using the frame index.
    frame = np.where(f_index==dpi)[0]
    frame = int(frame) #make sure it's an integer
    snake = snake[frame,:,:]
    
    ##Section 4: identifying index of any missing markers in the frame where the spline is being fit.
    if np.isnan(snake).any():
        x = np.where(np.isnan(snake))[0][0]
    else:
        x = np.nan
        
    if x==0:
        excluded.append(all_imported[i]['tn']) #can't calculate the torque effectively if the head marker is misisng.
        print(str(tn)+': Head is NaN')
    
    ## Section 5: determine torque
    else:       
        tval = torque(snake,spaces,svl,mg)
        torques.append(tval) #torques in X,Y,Z - newton meters.

        normed = np.sqrt(tval[0]**2+tval[1]**2)/(9.8*mg*svl) #normalize by mass and length.
        t_normed.append(normed)
        
        t_indices.append(frame)
        gap_svl.append(gap_size)
        sneks.append(ID)
        t_pos.append(sm_pos[i])
        trial.append(tn)
        gap_m.append(gs_m[i])
        beh_c.append(beh_code)

289: Mismatch between DPI and BC
283: Mismatch between DPI and BC
282: Mismatch between DPI and BC
281: Mismatch between DPI and BC
277: Mismatch between DPI and BC
276: Mismatch between DPI and BC
275: Mismatch between DPI and BC
94: Origin end not defined
93: Origin end not defined
85: Origin end not defined


In [10]:
gs_bin = [5 * round(i/5) for i in gap_svl]
tdata = pd.DataFrame(list(zip(trial,sneks,beh_c,gap_m,gap_svl,gs_bin,t_indices,t_normed,t_pos)),
                   columns =['trial', 'ID','Beh','gsm','gsr','gs_bin','TInd','TNorm','TPos'])

tdata['x_torq'] =[each[0] for each in torques]
tdata['y_torq'] =[each[1] for each in torques]
tdata['z_torq'] =[each[2] for each in torques]

tdata.head()

,Trial,Snake,Beh,gsm,gsr,gs_bin,TInd,TNorm,TPos,x_torq,y_torq,z_torq
0,274,85.0,1,0.622154,85.814306,85,378,0.025966,"[[[29.052959038533505, 58.66783431129612, -31....",-0.008356,0.004713,0.0
1,272,85.0,1,0.615736,84.929116,85,189,0.027637,"[[[79.73080467778203, 3.1201594521259244, -8.5...",0.003974,0.009278,0.0
2,271,85.0,1,0.615816,84.940108,85,710,0.026917,"[[[55.95856199581728, 29.005880728864014, -2.2...",-0.002528,0.009499,0.0
3,270,85.0,1,0.570038,78.625945,80,3126,0.011321,"[[[22.40558021185845, -38.76367608095813, 10.2...",0.000317,0.004122,0.0
4,269,85.0,1,0.571563,78.836340,80,1414,0.036897,"[[[21.053144734401204, 30.64708034742381, 44.2...",0.002947,0.013148,0.0


In [11]:
A,B = np.shape(tdata)
resultant = []
for i in np.arange(A):
    x = tdata['x_torq'][i]
    y = tdata['y_torq'][i]
    z = tdata['z_torq'][i]
    
    res = np.linalg.norm([x,y,z])
    resultant.append(res)
    
tdata['res'] = resultant

## Determine head position at transition as percent of gap size

In [12]:
hpt = [] #head position at transition as percent of gap size

A,B = np.shape(tdata)
for i in np.arange(A):
    if tdata['Beh'][i]==0:
        hpt.append(np.nan) #no transition frame for cantilevers
    else:
        gap = tdata['gsm'][i]
        posits = tdata['TPos'][i]
        posits = posits/1000.0 #mm to m
        inds = tdata['TInd'][i]
        head_p = np.linalg.norm(posits[inds,0,:])
        as_percent = head_p/gap*100
        hpt.append(as_percent)

In [13]:
tdata['hpt']=hpt

## Add extra parameters

In [14]:
A,B = np.shape(tdata)
svls = []
masses = []
for i in np.arange(A):
    trial = tdata['Trial'][i]
    index = trial_ns.index(trial)
    svl = all_imported[index]['svl']/100 #cm to m
    mass = all_imported[index]['mass']/1000 #g to kg
    svls.append(svl)
    masses.append(mass)
    
tdata['svl'] = svls
tdata['mass'] = masses

In [15]:
tdata.head()

,Trial,Snake,Beh,gsm,gsr,gs_bin,TInd,TNorm,TPos,x_torq,y_torq,z_torq,res,hpt,svl,mass
0,274,85.0,1,0.622154,85.814306,85,378,0.025966,"[[[29.052959038533505, 58.66783431129612, -31....",-0.008356,0.004713,0.0,0.009593,15.673883,0.725,0.0520
1,272,85.0,1,0.615736,84.929116,85,189,0.027637,"[[[79.73080467778203, 3.1201594521259244, -8.5...",0.003974,0.009278,0.0,0.010093,25.323470,0.725,0.0514
2,271,85.0,1,0.615816,84.940108,85,710,0.026917,"[[[55.95856199581728, 29.005880728864014, -2.2...",-0.002528,0.009499,0.0,0.009830,23.096026,0.725,0.0514
3,270,85.0,1,0.570038,78.625945,80,3126,0.011321,"[[[22.40558021185845, -38.76367608095813, 10.2...",0.000317,0.004122,0.0,0.004134,21.336266,0.725,0.0514
4,269,85.0,1,0.571563,78.836340,80,1414,0.036897,"[[[21.053144734401204, 30.64708034742381, 44.2...",0.002947,0.013148,0.0,0.013475,28.824730,0.725,0.0514


In [16]:
#save as csv to use in R
tsub = tdata.drop('TPos',axis=1) #not needed in stats analysis
tsub = tsub.drop('z_torq',axis=1) #not needed in stats analysis
tsub = tsub.drop('TInd',axis=1) #not needed in stats analysis
tsub = tsub[tsub.Beh != 0] #only studying transition torques for noncantilevers (cantilevers have no transition)
tsub = tsub.drop('Beh',axis=1) #since all trials in this dataset will be noncantilevers, don't need a behavior column

tsub.to_csv(cwd+'/R files/Summary Datasets/tdata.csv',index=False)